# Diving Into Presidential Speeches

*by [Charlie Windham](https://github.com/cwindham/)*

The 2016 presidential election promises to be one of the most interesting (to put it mildly) in recent memory.  In order to get some historical context, I thought it would be fun to analyze past presidential speeches. 
Here I want to dig into this data and answer a few questions:

- How has presidential rhetoric changed over time?
- Which presidents were the most positive?  The most negative?
- How do Clinton and Trump compare to past presidents?

If you're interested in how the plots and figures below were created, I have made available all the Python code I used to run this analysis. For details, see [the Github Repository](http://github.com/cwindham/PresidentialSpeeches) or follow the links below each figure.

# Dataset Overview

The data was obtained from two sources:

Presidential speech data came from the [Miller Center at the University of Virginia](http://millercenter.org/president/speeches).  The Miller Center speech archive represents a curated list of all of the most important speeches given by each president.

Candidate data for the 2016 election came from the [American Presidency Project at UCSB](http://www.presidency.ucsb.edu/2016_election.php).  Data was filtered to include only formal addresses.  Town halls, interviews, etc. were excluded.

For the script used to scrape and clean the data please see [this notebook](http://github.com/cwindham/PresidentialSpeeches/blob/master/get-speeches.ipynb).

Combined the dataset comprises almost 1000 speeches given by all 43 presidents and the 2 current major party 2016 nominees, ranging in date from 1789-2016.  As we will see, the characteristics of presidential speeches have changed significantly over this time.  Let's start by looking at some basic trend data:

## Are presidents speaking more simply than they used to?

![Alt text](files/figs/word_length_trend.png)
<small><i><a href="basic-analysis.ipynb#Word-Length-Trend">figure source</a></i></small>

![Alt text](files/figs/sentence_length_trend.png)
<small><i><a href="basic-analysis.ipynb#Sentence-Length-Trend">figure source</a></i></small>

Strikingly evident here is the sudden drop in average sentence length in the early 1920s.  The president during this time, Warren Harding, was the [first president to deliver a speech via radio](http://www.history.com/this-day-in-history/harding-becomes-first-president-to-be-heard-on-the-radio).

The word length trend isn't quite as clear, but still shows a gradual decline in word complexity over time.  Combined with the decrease in sentence length, this makes for an overall simplification of the presidential rhetoric.

I would posit that these trends represent a fundamental shift in presidential discourse from the written to spoken word.  It is much more difficult to parse long sentences and words in real time while listening. In contrast, while reading you can slow down or reread confusing language if necessary.

# Going Deeper with Natural Language Processing and Machine Learning

With these trends in mind, we can now ask some questions that require some more sophisticated modeling of the data.  Since we now suspect that shifts over time in the primary communication medium might cause changes in presidential speaking patterns, is there another way we can investigate this?

As luck would have it, yes!

## How has the evolution of communication media influenced presidential speaking styles?

One way we can answer this question is to use [*Unsupervised Machine Learning*](https://en.wikipedia.org/wiki/Unsupervised_learning) to  discover structure in the presidential speeches.  However, there is some housekeeping to do first.


Since the presidential speech data is in text format, we need to somehow standardize the data first before we can analyze it.  There is a well-established method for doing this type of work using [*Natural Language Processing*](https://en.wikipedia.org/wiki/Natural_language_processing).  My version goes something like this:
- Removing punctuation and lowercase the text.
- Tokenize by splitting the block text into its component words.
- Stem the tokens. Basically, this amounts to grouping words by their root and part of speech e.g. "running" and "ran" become "run", while "runny" and "runnier" become "runni".
- Remove stop words, words that are so common in the English language as to provide little relevant information e.g. pronouns like "I, you, he" and articles like "a, an, the".


Now that we have the words in a format that is easily digested, we need some way of counting and storing them.  We will use [*Feature Hashing*](https://en.wikipedia.org/wiki/Feature_hashing) here to improve performance (both speed and memory usage).  You can think of feature hashing as being similar to buying produce at the grocery store self-checkout; you can look up the produce by its name or picture, but entering the PLU code is much faster.  The PLU code in this analogy represents the hash index.  The process goes like this:

- Aggregate all of the 43 presidents' processed speeches into a corpus.
- Hash the tokens. This gives us a very sparse matrix (total matrix size was set to 2 \*\* 18 to avoid collisions) with the count of the token occurences at each hash index.

Once the speeches, have been hashed there is just one last processing step we need to do. To introduce the problem:  we are looking to *separate the presidents into groups* based on the words they use in their speeches.  This means that the word "America" is not as useful as the word "Iraq" since it appears much more commonly.  The method we will use to achieve this is called [*Term Frequency-Inverse Document Frequency (TF-IDF)*](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).  It goes like this:

- Fit a TF-IDF vectorizer on the hashed corpus. This weights the terms based on how frequently they appear in the corpus with less frequent words receiving a larger weight.
- Use the trained TF-IDF vectorizer to transform each president's aggregated speeches into a vector representing the most "significant" words used by each president.  "Significance" in this context represents a multiplication of a word's frequency by its learned weight in the TF-IDF vectorizer.


This next part is kind of abstract, but stay with me: there were ~33000 unique words in the presidential speeches, and each president used some subset of these words in their speeches.  In this way, each presidents words can be viewed as ***a single point in a 33000-dimensional space***.  Now, we can begin to query the relationships between presidents, using these 33000-dimensional points as a proxy. 

People are very good at visualizing two-dimensional or three-dimensional data. The plots above are two-dimensional (think "x" values vs. "y" values), but as the dimensionality grows, visualization becomes increasingly difficult if not impossible.  To better understand high-dimensional data, we can use dimensionality reduction algorithms. In this case, we would like to reduce the dimensions of the data from 33000 to 2, while maintaining an idea of the data structure.

The method we will use is [*Principal Component Analysis (PCA)*](https://en.wikipedia.org/wiki/Principal_component_analysis), which is a way of somehow flattening high-dimensional data to create a lower-dimensional projection while still preserving important relationships. Applying PCA to our presidential speech data gives the following plot:

![Alt text](files/figs/raw_pca.png)
<small><i><a href="tf-idf.ipynb#Raw-PCA">figure source</a></i></small>

Notable here is how the year increases as you proceed around the graph from left to right.  This supports the notion of time being the primary variable affecting presidential discourse (remember that how close the points are is somehow a measure of *similarity*).  This makes intuitive sense.  Language evolves and the hot-button political issues change.  Whats impressive here is that we have gleaned this information from just the transcripts of the speeches **without any *a priori* knowledge of date**.  Pretty cool!

To get a better intuition as to how to split this data, we might use another unsupervised machine learning method, a clustering algorithm, specifically [*Agglomerative Clustering*](https://en.wikipedia.org/wiki/Complete-linkage_clustering), to identify groups within the 33000-dimensional space and assign cluster membership.

Agglomerative is a "bottom-up" clustering method where each point starts in its own cluster.  Clusters are then recursively merged as the algorithm progresses based on some kind of distance metric.  In our case, we are interested in [*Cosine Similarity*](https://en.wikipedia.org/wiki/Cosine_similarity).  Cosine similarity performs quite highly in text comparison tasks, and in fact is the primary reason why we used agglomerative clustering - because it allows you to use non Euclidean distance metrics like cosine similarity.  We will use three clusters since we are interested in the effects of writing versus radio versus video being the primary communication medium.

After clustering, we can re-visualize the data with these cluster labels to show the trends within each group:

![Alt text](files/figs/lexical_similarity.png)
<small><i><a href="tf-idf.ipynb">figure source</a></i></small>

I'll take this moment to re-emphasize that the only input to our clustering algorithm was the processed words used by each president, nothing about date, sentence length, or anything else.  Given just that data, the algorithm does a pretty good job:
- Cluster 1 (blue): George Washington through William Taft
- Cluster 2 (red):  Woodrow Wilson through Dwight Eisenhower
- Cluster 3 (green):  JFK through Barack Obama


I have loosely labeled the clusters "written", "radio", and "video" respectively for reasons that will shortly become apparent:

All clusters ended up containing presidents in chronological order which I think is significant.

The presidential speeches from Washington to Taft were either delivered via written missives or survive as the official of the spoken speech (think the speech you write down versus the speech you actually give).

While Woodrow Wilson never spoke to the public via radio while president (his radio addresses came afterwards), he did hold the [first presidential press conference in history](https://www.whitehousehistory.org/presidential-press-conferences).  Wilson also presided over the [birth of the radio industry](https://en.wikipedia.org/wiki/History_of_radio).  As such, his presidency represented a fundamental shift in communication.  And, as we know from earlier, his successor Warren Harding was the first ever president to deliver a speech via radio.

JFK is regarded as being the [first "television" president](https://www.jfklibrary.org/JFK/JFK-in-History/John-F-Kennedy-and-the-Press.aspx), that is the first president to speak directly to the people without delay or editing.  His presidency was the first in which televisions were widespread in America from the start of his term.  The prvious two presidents, Truman and Eisenhower, appeared on television but are widely considered by historians to have been less effective at using the medium.  As such, they tend to get lumped in with the presidents before them both by scholars and this clustering algorithm.

We will now take a look at the modern "television" presidents, but first a primer on sentiment analysis:

# Analyzing What Makes an Effective Speaker with Sentiment Analysis

We all know a moving speech when we hear one.  But what exactly makes a speech good or not?  To answer this question we will be using [*Sentiment Analysis*](https://en.wikipedia.org/wiki/Sentiment_analysis).  Since we are interested in how speech sentiment changes over the course of the speech, we will be using an analysis technique known as the [*Bag-of-Words Model*](https://en.wikipedia.org/wiki/Bag-of-words_model).

In our case, each word is assigned a value from -1 to +1 based on how negative or positive the word is using [SentiWordNet](https://en.wikipedia.org/wiki/Bag-of-words_model) i.e. the word "good" might score as 0.5 while "bad" might score as -0.5.  SentiWordNet scales the sentiment score based on the magnitude of the intensity of the feeling associated with the word.  To reuse our example, "good" might be 0.5, while "outstanding" might score 0.9.  These scores are added up word-by-word to get a score at the sentence level.  These sentence scores are then normalized for total speech length and combined to get an overall score.  By adding each sentences score to the sum of scores for the previous sentences, we can get an idea of the overall mood of a speech at a given point.  The end result is a plot that looks like this:

![Alt text](files/figs/obama_jfk.png)
<small><i><a href="modern-analysis.ipynb#Sentiment-Comparison">figure source</a></i></small>

The first thing that jumps out at you is how much wider the range of JFK's sentiment scores are than Obama's.  Additionally, while the average speech (thick lines) of both presidents looks similar, the individual speeches (thin lines) of JFK have much more up-and-down movement than Obama.  Put more technically, the *means* of both president's speech sentiments are similar, but the *variance* of JFK's sentiment scores is larger.  This reflects a more emotive, dynamic speaking style and shows quantitatively why JFK is considered one of America's all time great orators.

Visualized as a boxplot:

![Alt text](files/figs/sentiment_boxplot.png)
<small><i><a href="modern-analysis.ipynb#Sentiment-Boxplot">figure source</a></i></small>

Here again we see that JFK's range of sentiments, sans outliers, is larger than that of any other modern president.

## Historical Presidential Sentiment

Let's now take a look at the most positive and negative presidents based on mean speech sentiment.

<center><strong>Most Negative</strong></center>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>president</th>
      <th>speech_sentiment</th>
    </tr>
    <tr>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>James Buchanan</th>
      <td>0.024200</td>
    </tr>
    <tr>
      <th>Herbert Hoover</th>
      <td>0.088211</td>
    </tr>
    <tr>
      <th>Andrew Johnson</th>
      <td>0.090909</td>
    </tr>
    <tr>
      <th>Abraham Lincoln</th>
      <td>0.103621</td>
    </tr>
    <tr>
      <th>Chester Arthur</th>
      <td>0.122199</td>
    </tr>
  </tbody>
</table>
<small><i><a href="modern-analysis.ipynb#Historical-Sentiment">figure source</a></i></small>

The most negative presidents are about as expected.  We have the three presidents from before, during, and after the Civil War (Buchanan, Lincoln, and Johnson respectively).  Hoover was president during the Great Depression.  The only surprise is Chester Arthur.

<center><strong>Most Positive</strong></center>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>speech_sentiment</th>
    </tr>
    <tr>
      <th>president</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>George Washington</th>
      <td>0.444237</td>
    </tr>
    <tr>
      <th>John Adams</th>
      <td>0.360260</td>
    </tr>
    <tr>
      <th>James Monroe</th>
      <td>0.350620</td>
    </tr>
    <tr>
      <th>John Quincy Adams</th>
      <td>0.327188</td>
    </tr>
    <tr>
      <th>Martin Van Buren</th>
      <td>0.318015</td>
    </tr>
  </tbody>
</table>
<small><i><a href="modern-analysis.ipynb#Historical-Sentiment">figure source</a></i></small>

The most positive presidents are likewise predictable, with the exception of Van Buren.  Washington unsurprisingly is number one. The rest of the top four are all Founding Fathers who basically served as America's cheerleaders during the early days, trying to convince the states of the virtues of the federal government.

# Bonus: Looking at the 2016 Major Party Presidential Candidates

Now, it's time for a look at the 2016 presidential candidates, Hillary Clinton and Donald Trump, to see where they fit in.

## Finding Similar Historical Presidents with K-Nearest Neighbors

Similar to the procedure used in the clustering algorithm from before, we can find presidents with similar speaking styles (this time using sentiment instead of word choice).  We will use the accumulated sentence-over-sentence sentiment of each speech, resampled to a standard length of 100.  We can then take the mean of these vectors to get a representative average of the speeches, just like we did in the Obama JFK comparison above.

Once we have the average speech vectors we can run a [*K-Nearest Neighbors*](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) search to find the presidents most similar to one another.  In our case we will look for the five most similar presidents to Trump or Clinton.

<center><strong>Donald Trump</strong></center>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>distance</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Donald Trump</th>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>Chester Arthur</th>
      <td>0.118293</td>
    </tr>
    <tr>
      <th>Andrew Johnson</th>
      <td>0.160548</td>
    </tr>
    <tr>
      <th>Herbert Hoover</th>
      <td>0.175594</td>
    </tr>
    <tr>
      <th>William Harrison</th>
      <td>0.199540</td>
    </tr>
    <tr>
      <th>James Buchanan</th>
      <td>0.297691</td>
    </tr>
  </tbody>
</table>
<small><i><a href="modern-analysis.ipynb#Nearest-Neighbors">figure source</a></i></small>

The nearest presidents to Trump should look familiar, most of them are from the list of the most negative presidents.  This matches with the overall negative tone of the Trump campaign.

<center><strong>Hillary Clinton</strong></center>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>distance</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Hillary Clinton</th>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>Martin Van Buren</th>
      <td>0.161003</td>
    </tr>
    <tr>
      <th>John Quincy Adams</th>
      <td>0.186880</td>
    </tr>
    <tr>
      <th>James Monroe</th>
      <td>0.201286</td>
    </tr>
    <tr>
      <th>James Garfield</th>
      <td>0.219908</td>
    </tr>
    <tr>
      <th>Andrew Jackson</th>
      <td>0.323196</td>
    </tr>
  </tbody>
</table>

<small><i><a href="modern-analysis.ipynb#Nearest-Neighbors">figure source</a></i></small>

In contast, Clinton is grouped with three of the most positive presidents.  This reflects the difference in how the respective candidates have chosen to run their campaigns.

The clusters in graph form using PCA for dimensionality reduction:

![Alt text](files/figs/candidate_clusters.png)
<small><i><a href="modern-analysis.ipynb#Candidate-Clusters">figure source</a></i></small>

To drive the point home, lets'd look at one final graph comparing the candidates to recent presidents:

![Alt text](files/figs/candidate_boxplot.png)
<small><i><a href="modern-analysis.ipynb#Candidate-Boxplot">figure source</a></i></small>

We can see that Trump speeches usually fall within the bottom quartile of most other presidents' speeches.  Interestingly, we can also see that Clinton has tried very hard to avoid a negative tone.  There is almost no difference between her bottom quartile and second quartile.  It's cool how data can so clearly show the difference between both campaigns' strategies!